In [24]:
import numpy as np
import scipy.stats as st
import pickle

 ## ES Calculation


In [25]:
# load dates of EREs
# ERE_start_days is a 400*1440 array, whose each element is a list of ERE dates of the corresponding point.
ERE_start_days = np.load('ERE_start_days_recompute.npy', allow_pickle = True)

# load threshold of ES for link construction
threshold = np.load('trmm7_mnoe78_thresholds_005_tm10_2_y19.npy', allow_pickle = True)

In [26]:
# the threshold is a triangular matrix, and we construct the whole matrix sig_level for efficiency in later ES calculation
# if one point has i EREs and another point has j EREs, then the threshold of link between them is sig_level[i, j]
sig_level = threshold + threshold.T
for i in range(sig_level.shape[0]):
    sig_level[i,i] /= 2

In [27]:
def SCA_index(lat_t, lon_t):
    # This function uses latitude and longitude to get 2d index of the data array
    # Input latitude and longgitude: lat_t is between -50 and 50, and lon_t is between -180 and 180
    # Output latitude index and longitude index: latindex between 0 and 400, and lonindex is between 0 and 1440
    index = (lat_t+50)*1440*4 + (180+lon_t)*4
    index = int(index)
    latindex = index // 1440
    lonindex = index % 1440
    return np.array([latindex, lonindex])
    

In [28]:
# get the 2d indices of the 25 points from SCA

SCA_list = np.array([])
for lat_t in [27.5, 27.75, 28., 28.25, 28.5]:
    for lon_t in [78.5, 78.75, 79., 79.25, 79.5]:
        SCA_list = np.concatenate((SCA_list, SCA_index(lat_t, lon_t)))
SCA_list = SCA_list.reshape((25,2))
SCA_list = np.array(SCA_list, dtype = 'int')

In [29]:
# get the 25 lists corresponding to ERE dates of the 25 points

SCA_time_series = []
for l in SCA_list:
    SCA_time_series.append(np.array(ERE_start_days[l[1]][l[0]]))

In [30]:
# flatten ERE_start_days here for ease in later loop; 
# transpose before flatten because KDE calculation will be done with respect to grid 400*1440, while ERE_start_days is an 1440*400 array
ERE_start_days_array_1d = (ERE_start_days.T).flatten()

In [31]:
def calc_ES_ij_array(day_list_i, day_list_j):
    # Input: two lists/arrays of ERE dates
    # Output: number of ES if ES between them is higher than the threshold; 0 otherwise (not attain the threshold)
    ES_ij = 0
    len_i, len_j = len(day_list_i), len(day_list_j)
    i, j = 0, 0
    while i < len_i and j < len_j:
        threshold_list_i = [day_list_i[i+1]-day_list_i[i] if i < len_i-1 else np.inf, day_list_i[i]-day_list_i[i-1] if i > 0 else np.inf]
        threshold_list_j = [day_list_j[j+1]-day_list_j[j] if j < len_j-1 else np.inf, day_list_j[j]-day_list_j[j-1] if j > 0 else np.inf]
        tau_1 = min(min(threshold_list_i), min(threshold_list_j)) / 2
        tau = min(tau_1, 10)  # tau_2=10
        if abs(day_list_i[i] - day_list_j[j]) < tau:
            ES_ij += 1
            i += 1
            j += 1
        elif day_list_i[i] < day_list_j[j]:
            i += 1
        else:
            j += 1
    if ES_ij > sig_level[len_i][len_j]:
        return ES_ij
    else:
        return 0

In [32]:
# This cell and the following four cells are for finding the points which have significant number of ES to the SCA 25 points
# The reason why I split the 25 points into 5 pieces is that the function np.vstack takes much time for large arrays
# and doing calculation in 5 pieces then stacking them together takes much less time than do all 25 points together
# These 5 cells should be done in 40 min. 
# You can run them again or just load the one we computed before, named "SCA_links.npy", in significant link calculation.
SCA_links1 = np.array([0,0,0])
for i in range(5):
    for j in range(576000):
        SCA_point = SCA_time_series[i]
        world_point = ERE_start_days_array_1d[j]
        if world_point == 0:
            continue
        elif len(world_point) < 3:
            continue
        else:
            world_point = np.array(world_point, dtype = 'int')
            ES_ij = calc_ES_ij_array(SCA_point, world_point) 
            if ES_ij > 0:
                SCA_links1 = np.vstack((SCA_links1, np.array([i, j, ES_ij])))
    print(i)

0
1
2
3
4


In [33]:
SCA_links2 = np.array([0,0,0])
for i in range(5, 10):
    for j in range(576000):
        SCA_point = SCA_time_series[i]
        world_point = ERE_start_days_array_1d[j]
        if world_point == 0:
            continue
        elif len(world_point) < 3:
            continue
        else:
            world_point = np.array(world_point, dtype = 'int')
            ES_ij = calc_ES_ij_array(SCA_point, world_point)
            if ES_ij > 0:
                SCA_links2 = np.vstack((SCA_links2, np.array([i, j, ES_ij])))
    print(i)

5
6
7
8
9


In [34]:
SCA_links3 = np.array([0,0,0])
for i in range(10, 15):
    for j in range(576000):
        SCA_point = SCA_time_series[i]
        world_point = ERE_start_days_array_1d[j]
        if world_point == 0:
            continue
        elif len(world_point) < 3:
            continue
        else:
            world_point = np.array(world_point, dtype = 'int')
            ES_ij = calc_ES_ij_array(SCA_point, world_point)
            if ES_ij > 0:
                SCA_links3 = np.vstack((SCA_links3, np.array([i, j, ES_ij])))
    print(i)

10
11
12
13
14


In [36]:
SCA_links4 = np.array([0,0,0])
for i in range(15, 20):
    for j in range(576000):
        SCA_point = SCA_time_series[i]
        world_point = ERE_start_days_array_1d[j]
        if world_point == 0:
            continue
        elif len(world_point) < 3:
            continue
        else:
            world_point = np.array(world_point, dtype = 'int')
            ES_ij = calc_ES_ij_array(SCA_point, world_point)
            if ES_ij > 0:
                SCA_links4 = np.vstack((SCA_links4, np.array([i, j, ES_ij])))
    print(i)

15
16
17
18
19


In [37]:
SCA_links5 = np.array([0,0,0])
for i in range(20, 25):
    for j in range(576000):
        SCA_point = SCA_time_series[i]
        world_point = ERE_start_days_array_1d[j]
        if world_point == 0:
            continue
        elif len(world_point) < 3:
            continue
        else:
            world_point = np.array(world_point, dtype = 'int')
            ES_ij = calc_ES_ij_array(SCA_point, world_point)
            if ES_ij > 0:
                SCA_links5 = np.vstack((SCA_links5, np.array([i, j, ES_ij])))
    print(i)

20
21
22
23
24


In [38]:
SCA_links = np.vstack((SCA_links1[1:],SCA_links2[1:],SCA_links3[1:],SCA_links4[1:],SCA_links5[1:]))
np.save('SCA_links_Ray.npy', SCA_links)